# Программирование на Python
## Семинар 3. Повторение основ
#### Задача с прошлого занятия
Мы хотим автоматизировать процесс перекладки текстовых файлов с данными из хранилища в БД (Postgres). Предположим, что таблицы каждый раз разные. Соответственно, перед записью данных необходимо создать таблицу. Это можно сделать примерно так:

```
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);
```

PostgreSQL [располагает](https://www.postgresql.org/docs/current/datatype.html) следующими типами данных (это не все, что есть, но все, что нам сейчас пригодится):

<table>
    <tr>
        <th>Python</th>
        <th>PostgreSQL</th>
    </tr>
    <tr>
        <td>int</td>
        <td>smallint, integer, bigint</td>
    </tr>
    <tr>
        <td>float</td>
        <td>real</td>
    </tr>
    <tr>
        <td>bool</td>
        <td>boolean</td>
    </tr>
    <tr>
        <td>str</td>
        <td>varchar, text</td>
    </tr>
</table>

Реализуйте соответствующую функцию. Она должна иметь следующие аргументы:

- `filename` (строка) - путь к текстовому файлу;
- `sep` (строка, по дефолту для CSV) - разделитель;
- `tablename` (строка, по дефолту соответствует названию файла без расширения);
- `varnames` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`);
- `dtypes` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`).

Функция должна записывать получившийся скрипт в файл 'create_table.sql' и возвращать путь к этому файлу.

In [ ]:
# надо проверять на bool
# надо проверять на int -> smallint, integer, bigint
# надо проверять на float
# все, что осталось в строки

In [55]:
big_list = list(range(100000))
big_set = set(range(100000))

In [56]:
%%timeit

_ = 50000 in big_list

743 µs ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [57]:
%%timeit

_ = 50000 in big_set

56.1 ns ± 3.53 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [58]:
all([True, True])

True

In [59]:
all([1, 1])

True

In [43]:
some_zip = zip(cnames, dts)

In [27]:
some_zip

In [28]:
some_zip_iter = iter(some_zip)

In [37]:
next(some_zip_iter)

StopIteration: 

In [44]:
for n, d in some_zip:
    print(f'dtype for column {n} is {d}')

dtype for column gender is text
dtype for column race/ethnicity is text
dtype for column parental level of education is text
dtype for column lunch is text
dtype for column test preparation course is text
dtype for column math score is integer
dtype for column reading score is integer
dtype for column writing score is integer


In [41]:
some_zip

In [56]:
import re

# {0,1} == ?
# {0,} == *
# {1,} == +

trash_symbols = '" \n'
pattern_int = re.compile('-{0,1}\d+')
pattern_float = re.compile('-{0,1}\d*\.\d*')
template = 'CREATE TABLE {tablename} (\n{pairs}\n);'

def check_dtype(vector, fun) -> bool:
    # strict rule
    return all(fun(elem) for elem in vector)

def check_int(value: str) -> bool:
    return pattern_int.fullmatch(value) is not None

def check_float(value: str) -> bool:
    return pattern_float.fullmatch(value) is not None

def check_bool(value: str) -> bool:
    return value.lower() in {'true', 'false'}

def guess_type(vector) -> str:
    if check_dtype(vector, check_bool):
        return 'boolean'
    elif check_dtype(vector, check_int):
        return 'integer'
    elif check_dtype(vector, check_float):
        return 'float'
    else:
        return 'text'

def tablename_from_path(path):
    slash_index = path.rfind('/')
    dot_index = path.rfind('.')

    return path[slash_index + 1:dot_index]

def sql_from_file(
    filename: str,
    fileoutname: str,
    sep=',',  # sep: str = ','
    tablename=None,
    varnames=None,
    dtypes=None
) -> str:
    # read file
    with open(filename, 'r') as file:  # file = open(path, 'r')
        lines = []
        
        for line in file:  # iter(file) -> next
            line_split = line.split(sep)
            line_clean = [elem.strip(trash_symbols) for elem in line_split]
            
            lines.append(line_clean)

    # guess tablename
    if tablename is None:
        tablename = tablename_from_path(filename)

    # guess varnames
    if varnames is None:
        varnames = lines[0].copy()  # list(lines[0])

    # transpose lines
    lines_t = zip(*lines[1:])
    
    # guess dtypes
    if dtypes is None:
        dtypes = [guess_type(v) for v in lines_t]

    # create sql-request
    pairs = ',\n'.join([f'    "{name}" {dtype}' for name, dtype in zip(varnames, dtypes)])
    sql_request = template.format(tablename=tablename, pairs=pairs)

    # write sql-request
    with open(fileoutname, 'w') as file:
        file.write(sql_request)

    return fileoutname

In [57]:
path = '../Занятие 1/StudentsPerformance.csv'
sqlpath = 'create_table.sql'

_ = sql_from_file(
    path,
    sqlpath
)

In [50]:
pairs = ',\n'.join([f'    "{name}" {dtype}' for name, dtype in zip(cnames, dts)])

In [52]:
template = 'CREATE TABLE {tablename} (\n{pairs}\n);'
print(template.format(tablename=tname, pairs=pairs))

CREATE TABLE StudentsPerformance (
    "gender" text,
    "race/ethnicity" text,
    "parental level of education" text,
    "lunch" text,
    "test preparation course" text,
    "math score" integer,
    "reading score" integer,
    "writing score" integer
);


In [ ]:
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);

In [10]:
check

('StudentsPerformance',
 ['gender',
  'race/ethnicity',
  'parental level of education',
  'lunch',
  'test preparation course',
  'math score',
  'reading score',
  'writing score'],
 ['text', 'text', 'text', 'text', 'text', 'integer', 'integer', 'integer'])

In [8]:
trash_symbols = '" \n'

with open(path, 'r') as file:  # file = open(path, 'r')
    lines = []
    
    for line in file:  # iter(file) -> next
        line_split = line.split(',')
        line_clean = [elem.strip(trash_symbols) for elem in line_split]
        
        lines.append(line_clean)

In [ ]:
for elem in sequence:  # для чего, где
    do_smth(elem)  # сделать что

[do_smth(elem) for elem in sequence]  # сделать что, для чего, где

In [7]:
[elem.strip(trash_symbols) for elem in lines[0]]

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

In [20]:
{(-1) ** i for i in range(10) if i % 2 == 0}

{1}

In [17]:
((-1) ** i for i in range(10),)

SyntaxError: invalid syntax (1997625108.py, line 1)

In [22]:
words = ['word', 'wooord', 'wooooooord', 'wrd']

wdict = {}

for word in words:
    wdict[word] = len(set(word))  # wdict.update({word: len(set(word))})

wdict

{'word': 4, 'wooord': 4, 'wooooooord': 4, 'wrd': 3}

In [24]:
wdict = {word: len(set(word)) for word in words}
wdict

{'WORD': 4, 'WOOORD': 4, 'WOOOOOOORD': 4, 'WRD': 3}

In [6]:
' "gender"a'.strip('" a')

'gender'

#### Задача 1

Известно, что подготовка и исследование данных для машинного обучения занимают куда больше времени, чем собственно машинное обучение. В частности, существует такая процедура, как создание т. н. dummy-переменных.

![dummy](https://www.statology.org/wp-content/uploads/2021/02/dummyvartrap1-768x344.png)

В `pandas` уже есть функция, которая принимает на вход текстовую колонку таблицы и возвращает вместо нее много колонок с dummy-переменными. Однако проблема в том, что в ваших данных есть переменные, значения которых представляют из себя словосочетания со знаками препинания (количество таких слов формально не ограничено). Некоторые алгоритмы "любят" только простые названия переменных вида `variable` или `simple_variable`. Кроме того, так или иначе для последующей работы неплохо было бы стандартизировать все названия.

Поэтому вам нужно создать функцию, которая бы принимала на вход список из значений переменной и производила следующую предобработку всех значений в колонке:

- удаление всех знаков препинания;
- приведение всех букв к нижнему регистру;
- замену всех пробелов на нижнее подчеркивание;
- ограничение длины каждого слова первыми четырьмя буквами.

Функция должна возвращать словарь (т. н. mapper), по которому можно преобразовать переменную в новый вид (с помощью методов `.map()` / `.apply` - о них вам расскажут позднее). Ключами должны быть старые значения, а собственно значениями - новые.

Не всегда бывает так, что значения переменной написаны на латинице. Напишите вашу функцию таким образом, чтобы по дефолту она обрабатывала англоязычные переменные, но также имела возможность (при соответствующем значении аргумента) обработать и кириллические. Вам может пригодиться функция `translit` из модуля `transliterate` (пример работы с кириллицей можете найти ниже).

**Пример**

```
function input:
['Agree', 'Agree', 'Neither agree nor disagree', "Don't know", 'Neither agree nor disagree', 'Neither agree nor disagree', 'Disagree', 'No answer', 'Agree strongly', 'No answer', 'Agree', 'Refusal', 'Refusal', 'Disagree strongly', 'Disagree']

function return:
{
    'Agree strongly': 'agre_stro',
    'Agree': 'agre',
    'Neither agree nor disagree': 'neit_agre_nor_disa',
    'Disagree': 'disa',
    'Disagree strongly': 'disa_stro',
    'Refusal': 'refu',
    "Don't know": 'dont_know',
    'No answer': 'no_answ'
}
```

In [80]:
import re

from string import punctuation

punctuation = f'[{re.escape(punctuation.replace("_", ""))}]'
punctuation_re = re.compile(punctuation)

def fix_string(string: str) -> str:
    string = translit(string, 'ru', reversed=True)
    string = punctuation_re.sub('', string).lower()

    return '_'.join([word[:4] for word in string.split()])

def get_mapper(vector) -> dict:
    return {string: fix_string(string) for string in set(vector)}

In [81]:
check = ['Agree', 'Agree', 'Neither agree nor disagree', "Don't know", 'Neither agree nor disagree', 'Neither agree nor disagree', 'Disagree', 'No answer', 'Agree strongly', 'No answer', 'Agree', 'Refusal', 'Refusal', 'Disagree strongly', 'Disagree']

In [82]:
get_mapper(check)

{'Disagree': 'disa',
 'Agree': 'agre',
 'Disagree strongly': 'disa_stro',
 'Refusal': 'refu',
 'Agree strongly': 'agre_stro',
 "Don't know": 'dont_know',
 'Neither agree nor disagree': 'neit_agre_nor_disa',
 'No answer': 'no_answ'}

In [79]:
[word[:4] for word in example.split()]

['neither', 'agree', 'nor', 'disagree']

In [71]:
print(punctuation)

[!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^`\{\|\}\~]


In [59]:
punctuation_re = re.compile('\W')

In [61]:
punctuation_re.search('_,.-')

<re.Match object; span=(1, 2), match=','>

In [83]:
# пример работы функции translit

from transliterate import translit

print(translit('Полностью не согласен', 'ru', reversed=True))

Polnost'ju ne soglasen


In [ ]:
# наш код здесь

#### Задача 2
Предположим, что перед нами стоит задача создать dummy-переменные в PostgreSQL. К сожалению, это не `pandas`, и готовой функции на этот случай не предусмотрено. Однако вы знаете, что можно создавать новые переменные базируясь на значении старых используя конструкцию case...when...then...end:

```
PostgreSQL:
CASE WHEN <condition> THEN <value1> ELSE <value2> END AS <variable_name>
В ТОМ СЛУЧАЕ КОГДА <условие> ТОГДА <значение1> ИНАЧЕ <значение2> КОНЕЦ НАЗВАТЬ <имя_переменной>

Python:
if <condition>:
    <value1>
else:
    <value2>
```

Если категорий немного, то написать такой код несложно. Однако что делать, если их, к примеру, 100? В этом случае придется писать `CASE WHEN` столько раз, сколько уникальных значений содержит ваша переменная, да еще и придумывать каждый раз соответствующее название.

Используйте свои знания Python, чтобы автоматизировать процесс написания SQL-запроса. Напишите соответствующую функцию. Для автоматизированной генерации названия переменной используйте наработки предыдущей задачи. Учтите, что значения переменной не обязательно будут написаны на латинице.

На выходе функция должна печатать (в таком же формате, как в примере ниже) законченный блок SQL-запроса.

**Пример**
```
function return:
    CASE WHEN gincdif = 'Agree strongly' THEN 1 ELSE 0 END AS agre_stro,
    CASE WHEN gincdif = 'Agree' THEN 1 ELSE 0 END AS agre,
    CASE WHEN gincdif = 'Neither agree nor disagree' THEN 1 ELSE 0 END AS neit_agre_nor_disa,
    CASE WHEN gincdif = 'Disagree' THEN 1 ELSE 0 END AS disa,
    CASE WHEN gincdif = 'Disagree strongly' THEN 1 ELSE 0 END AS disa_stro,
    CASE WHEN gincdif = 'Refusal' THEN 1 ELSE 0 END AS refu,
    CASE WHEN ginsdif = "Don't know" THEN 1 ELSE 0 END AS dont_know,
    CASE WHEN ginsdif = "No answer" THEN 1 ELSE 0 END AS no_answ
```

In [ ]:
# наш код здесь